# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [108]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [109]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.head(10)


,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
0,0,Severo-Kuril'sk,50.6789,156.1250,28.17,86,100,5.91,RU,1644096582
1,1,Tuatapere,-46.1333,167.6833,53.82,82,99,3.80,NZ,1644096583
2,2,Busselton,-33.6500,115.3333,65.05,89,20,6.62,AU,1644096582
3,3,Vila Velha,-20.3297,-40.2925,78.75,85,40,7.00,BR,1644096308
4,4,Keti Bandar,24.1447,67.4497,69.94,61,0,9.08,PK,1644096583
5,5,Alice Springs,-23.7000,133.8833,56.75,88,0,0.00,AU,1644096584
6,6,Ushuaia,-54.8000,-68.3000,55.06,71,40,23.02,AR,1644096269
7,7,Nelson Bay,-32.7167,152.1500,73.00,63,93,4.00,AU,1644096584


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [110]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [111]:
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]].astype(float)
humidity = cities_df["humidity"].astype(float)

In [112]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
# creating new DF for ideal weather conditions
ideal_weather_df = cities_df.loc[(cities_df['max temp'] >= 55) &
                        (cities_df['max temp'] < 95) &
                        (cities_df['wind speed'] < 50) &
                        (cities_df['cloudiness'] < 50) &
                        (cities_df['humidity'] < 90)]
ideal_weather_df

,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
2,2,Busselton,-33.6500,115.3333,65.05,89,20,6.62,AU,1644096582
3,3,Vila Velha,-20.3297,-40.2925,78.75,85,40,7.00,BR,1644096308
4,4,Keti Bandar,24.1447,67.4497,69.94,61,0,9.08,PK,1644096583
5,5,Alice Springs,-23.7000,133.8833,56.75,88,0,0.00,AU,1644096584
6,6,Ushuaia,-54.8000,-68.3000,55.06,71,40,23.02,AR,1644096269


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
# from 6-APIs, class 3, 05-Stu_Google_Complex
hotel_df = pd.DataFrame(ideal_weather_df, columns=['city', 'country', 'lat', 'lon'])

hotel_df['hotel name'] = ""
hotel_df

,city,country,lat,lon,hotel name
2,Busselton,AU,-33.6500,115.3333,
3,Vila Velha,BR,-20.3297,-40.2925,
4,Keti Bandar,PK,24.1447,67.4497,
5,Alice Springs,AU,-23.7000,133.8833,
6,Ushuaia,AR,-54.8000,-68.3000,


In [115]:
#copied directly from 6-APIs, class 3, 05-Stu_Google_Complex

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    print(f"retrieving results for {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    results = response['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"the closest hotel is {results [0] ['name']}.")
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("--------------")

retrieving results for 2: Busselton.
the closest hotel is Observatory Guest House.
--------------
retrieving results for 3: Vila Velha.
the closest hotel is Quality Suites Vila Velha.
--------------
retrieving results for 4: Keti Bandar.
the closest hotel is Haji laloo memon house.
--------------
retrieving results for 5: Alice Springs.
the closest hotel is Desert Palms Alice Springs.
--------------
retrieving results for 6: Ushuaia.
the closest hotel is Albatross Hotel.
--------------


In [116]:
hotel_df

,city,country,lat,lon,hotel name
2,Busselton,AU,-33.6500,115.3333,Observatory Guest House
3,Vila Velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
4,Keti Bandar,PK,24.1447,67.4497,Haji laloo memon house
5,Alice Springs,AU,-23.7000,133.8833,Desert Palms Alice Springs
6,Ushuaia,AR,-54.8000,-68.3000,Albatross Hotel


In [117]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""

locations = hotel_df[['lat', 'lon']]
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
            locations, fill_color='rgba(0, 150, 0, 0.4)',
            stroke_color='rgba(0, 150, 0, 0.4)', scale=2,
            info_box_content=hotel_info)

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add marker and hotel layers ontop of heat map
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))